# import pycromanager

In [1]:
from pycromanager import Core, Studio
from smartpath_libraries.sp_acquisition import SPAcquisition

In [2]:
def init_pycromanager():
    from pycromanager import Core, Studio
    core = Core()
    studio = Studio()
    core.set_timeout_ms(20000)
    return core, studio
core,studio = init_pycromanager()

# core functions

In [3]:
core.set_auto_shutter(True)

## get stage position

In [5]:
stage_name = core.get_xy_stage_device()
xy = core.get_xy_stage_position()
stage_name, xy.x, xy.y

('XYStage:XY:31', 18944.77, 13749.01)

## check if live mode on

In [ ]:
if core.is_sequence_running():
    studio.live().set_live_mode(False)

## yaml loader

In [7]:
import yaml
config = yaml.load(open("config.yaml", "r"), Loader=yaml.FullLoader)
config = {**config["user_config"], **config["model_config"], **config["hard_config"]}
config

{'exposure-level': 'high',
 'snr-level': 'low',
 'autofocus-speed': 3,
 'lsm-resolution': 256,
 'lsm-bin-factor': 15,
 'lsm-scan-rate': '250000.0000',
 'lsm-pc-power': 0.4,
 'lsm-pmt-gain': 0.4,
 'slide-box': [-100, 600, 25500.0, 17000.0],
 'lsm-enhancer': None,
 'bf-enhancer': None,
 'classifier': None,
 'adv-interval': 3,
 'average-factor': 50,
 'batch-size': 8,
 'blindspot-rate': 0.05,
 'cnn-base-channel': 8,
 'dataset': 'datasets/PA_2081b',
 'gan-lambda': 5,
 'gpu': True,
 'image-channel': 1,
 'image-extension': 'tif',
 'iter-per-epoch': 200,
 'iterations': 200000,
 'learning-rate': 0.0001,
 'load-weights': 'supervised2022-08-07_13-02-52',
 'loss-gain': 1000,
 'norm-range': [5420, 44017],
 'norm-range-target': [6265, 38567],
 'only-pixel-loss': False,
 'percep-lambda': 0.5,
 'resolution': 512,
 'test-interval': 5,
 'threads': 0,
 'up-scale-factor': 2,
 'pixel-size-bf-20x': 0.222,
 'pixel-size-bf-4x': 1.105,
 'pixel-size-shg-base': 0.509,
 'pixel-size-shg': 0.509,
 'camera-resolutio

## background images

In [8]:
brightfield_4x_background_fname = (
    "data/presets/BG_4x.tiff"  # give a default 4x background image
)
brightfield_20x_background_fname = (
    "data/presets/BG_20x.tiff"  # give a default 20x background image
)


## check coordinates within limits

In [57]:
from dataclasses import dataclass, field

@dataclass
class sp_microscope_settings:
    x_range : list
    y_range : list
    z_range: list = field(default=None)
    f_range: list = field(default=None)
    
    focus_device: str = field(default = 'ZStage:Z:32')
    
camm = sp_microscope_settings([0, 40000.0],[0, 30000.0])
camm

sp_microscope_settings(x_range=[0, 40000.0], y_range=[0, 30000.0], z_range=None, f_range=None, focus_device='ZStage:Z:32')

In [86]:
#smartpath.mm_object_to_list(core.get_loaded_devices())


In [77]:
@dataclass
class sp_position:
    x: float
    y: float
    z: float = field(default=None)
    f: float = field(default=None)
    o: float = field(default=None)
    

coordinate1 = sp_position(50,100,20)
coordinate1

sp_position(x=50, y=100, z=20, f=None, o=None)

sp_position(x=18944.77, y=13749.01, z=-10511.99, f=None, o=None)

In [36]:
import numpy as np

In [128]:
import warnings

class smartpath:

    def __init__(self,core):
        self.core = core

    @staticmethod
    def is_coordinate_in_range(settings, position) -> bool:
        
        _within_ylimit = _within_xlimit = False
        if settings.x_range[0] < position.x < settings.x_range[1]:
            _within_xlimit = True
        else:
            warnings.warn(f" {position.x} out of range X {settings.x_range}")
        if settings.y_range[0] < position.y < settings.y_range[1]:
            _within_ylimit = True
        else:
            warnings.warn(f" {position.y} out of range Y {settings.y_range}")

        if not position.z:
            return _within_xlimit and _within_ylimit
        if settings.z_range:      
            _within_zlimit = False    
            if settings.z_range[0] < position.z < settings.z_range[1]:
                _within_zlimit = True
            else:
                warnings.warn(f" {position.z} out of range Y {settings.z_range}")            
            return _within_xlimit and _within_ylimit and _within_zlimit
        else:
            warnings.warn(f" Z range undefined : {settings.z_range}") 
            return False
            
    @staticmethod
    def mm_object_to_list(name):
        name = [name.get(i) for i in range(name.size())]
        return name

    
    def print_mm_status(self):
        status_str=""
        for device_name in smartpath.mm_object_to_list(self.core.get_loaded_devices()):
            
            device_property_names = self.core.get_device_property_names(device_name) 
            status_str = status_str + device_name +"\n"
            #+"::"+' '.join(smartpath.mm_object_to_list(device_property_names)) +"\n"
            
            property_names = smartpath.mm_object_to_list(device_property_names) 
            for prop in property_names: 
                val = self.core.get_property(device_name, prop) 
                status_str = status_str + prop+":"+ val +"\n"
            status_str = status_str +"----------------------------\n"
        return status_str

    @staticmethod
    def get_current_position(core):
        current_pos = sp_position(core.get_x_position(), core.get_y_position(), core.get_position())
        return current_pos
    
            
coordinate1.z = 100
smartpath.is_coordinate_in_range(camm,coordinate1)

C:\Users\lociuser\AppData\Local\Temp\ipykernel_15132\1245587835.py:31: UserWarning:  Z range undefined : None
  warnings.warn(f" Z range undefined : {settings.z_range}")


False

In [126]:
#sp = smartpath(core)
#print(sp.print_mm_status())

In [52]:
if  camm.z_range:
    print(camm)

In [41]:
camm.z_range= [0,10]

In [ ]:

## Bins utils

In [31]:
from skimage import img_as_float, exposure

class binli_utils:

    def __init__(self):
        pass

    @staticmethod
    def white_balance(img, bg, gain=1.0):
        img = img_as_float(img)
        bg = img_as_float(bg)
        r = np.mean(bg[:, :, 0])
        g = np.mean(bg[:, :, 1])
        b = np.mean(bg[:, :, 2])
        mm = max(r, g, b)
        img[:, :, 0] = np.clip(img[:, :, 0] * mm / r * gain, 0, 1)
        img[:, :, 1] = np.clip(img[:, :, 1] * mm / g * gain, 0, 1)
        img[:, :, 2] = np.clip(img[:, :, 2] * mm / b * gain, 0, 1)
        return img

    @staticmethod
    def flat_field(img, bg, gain=1):
        img = img_as_float(img)
        bg = img_as_float(bg)
        r = np.mean(bg[:, :, 0])
        g = np.mean(bg[:, :, 1])
        b = np.mean(bg[:, :, 2])
        img[:, :, 0] = 1 * exposure.rescale_intensity(np.clip(np.divide(img[:, :, 0], bg[:, :, 0] + 0.00) * r * gain, 0, 1), in_range=(0, 0.95), out_range=(0, 1))
        img[:, :, 1] = 1 * exposure.rescale_intensity(np.clip(np.divide(img[:, :, 1], bg[:, :, 1] + 0.00) * g * gain, 0, 1), in_range=(0, 0.95), out_range=(0, 1))
        img[:, :, 2] = 1 * exposure.rescale_intensity(np.clip(np.divide(img[:, :, 2], bg[:, :, 2] + 0.00) * b * gain, 0, 1), in_range=(0, 0.95), out_range=(0, 1))
        return img

    @staticmethod
    def is_background(img, t=0.3, tt=0.32):
        patch_h = int(img.shape[0]/8)
        patch_w = int(img.shape[1]/8)
        img = color.rgb2hsv(img)
        img_windows = np.squeeze(view_as_windows(img, (patch_h, patch_w, 3), step=(patch_h, patch_w, 3)))
        img_windows = np.reshape(img_windows, (img_windows.shape[0]*img_windows.shape[1], patch_h, patch_w, 3)) # nxm, 256, 256, 3
        img_max = np.max(img_windows, axis=0) # 256x256x3
        img_min = np.min(img_windows, axis=0) # 256x256x3
        sat_img = img_max[:, :, 1]
        bright_img = 1-img_min[:, :, 2]
        ave_sat = np.sum(sat_img)/(patch_h*patch_w)
        ave_bright = np.sum(bright_img)/(patch_h*patch_w)
        return ave_sat < t and ave_bright < tt


pixels = binli_utils.white_balance(pixels, background_image)
pixels = binli_utils.flat_field(pixels, bg_img)

In [60]:
position = coordinate1
settings = camm

In [71]:
core.get_x_position(), core.get_y_position(), core.get_position()

(18944.77, 13749.01, -10511.99)

In [61]:
position, settings

(sp_position(x=50, y=100, z=100, f=30, o=None),
 sp_microscope_settings(x_range=[0, 40000.0], y_range=[0, 30000.0], z_range=None, f_range=None, focus_device='ZStage:Z:32'))

In [131]:
current_pos = smartpath.get_current_position(core)
current_pos

sp_position(x=18944.77, y=13749.01, z=-10511.99, f=None, o=None)

In [137]:

def move_to_position(position,settings):
    # check position in range
    if smartpath.is_coordinate_in_range(settings,position):
        # verify focus device: bcs user can change it to F mode
        if core.get_focus_device() != settings.focus_device:
            warnings.warn(f" Changing focus device {core.get_focus_device()}")    
            core.set_focus_device(settings.focus_device)          
        # movement
        core.set_position(position.z)
        core.set_xy_position(position.x, position.y)
        core.wait_for_device(core.get_xy_stage_device())
        core.wait_for_device(core.get_focus_device())
    else:
        warnings.warn(" Move Cancelled")
        
new_pos = sp_position(current_pos.x+100,current_pos.y+100,current_pos.z+10)
move_to_position(new_pos,camm)
print(new_pos)
print(smartpath.get_current_position(core))

sp_position(x=19044.77, y=13849.01, z=-10501.99, f=None, o=None)
sp_position(x=19044.879, y=13848.95, z=-10502, f=None, o=None)


In [136]:
camm.z_range= [-10600,-10400]

In [ ]:
acq_id = len(glob.glob(os.path.join(save_path, acq_name+"*")))
acq_path = os.path.join(save_path, acq_name+"_{}".format(acq_id+1))
os.makedirs(acq_path, exist_ok=True)

bg_flag = False
sp_flag = False
bg_t = 0.28
bg_tt = 0.28

returns = {}
background_image = brightfield_20x_background_fname, brightfield_4x_background_fname

set_z_position
        
support_points = [(99999999, 99999999)] # dummy support point
support_focus = [pos_z]

if position_list.shape[1] == 3:
    tile_count = 0
    z_positions=np.ones(position_list.shape[0]) * core.get_position()
    
    for pos in range(position_list.shape[0]):
        sp_flag = False
                
                if focus_dive and mag=='4x':
                    support_distance = config['pixel-size-bf-4x'] * config['camera-resolution'][1] * config['autofocus-speed']
                    idx, min_distance = distance((x_pos, y_pos), support_points)
                    if min_distance <= support_distance:
                        pos_z = support_focus[idx]
                        pos_z = limit_stage(pos_z, (config['hard-limit-z'][0], config['hard-limit-z'][1]), default=config['Z-stage-4x'])
                        core.set_position(pos_z)
                        core.wait_for_device(z_device)
                        pixels = self.snap_image(rgb=True, flip_channel=True)
                        bg_flag = is_background(pixels, t=bg_t, tt=bg_tt)
                    else:
                        pos_z, pixels, bg_flag = self.autofocus(mag='4x', rgb=True, search_range=160, steps=5, snap=True, preset=z_pos) # snap at top but return center z
                        if bg_flag:
                            if len(support_points)>=2:
                                pos_z = limit_stage(pos_z, (config['hard-limit-z'][0], config['hard-limit-z'][1]), default=config["Z-stage-4x"])
                                core.set_position(pos_z)
                            else:
                                pos_z = z_pos
                            core.wait_for_device(z_device)
                            pixels = self.snap_image(rgb=True, flip_channel=True)
                        else:
                            support_points.append((x_pos, y_pos))
                            support_focus.append(pos_z)
                            sp_flag = True
                    z_positions[pos] = pos_z
                           
                if focus_dive and mag=='20x':
                    support_distance = config['pixel-size-bf-20x'] * config['camera-resolution'][1] * config['autofocus-speed']
                    idx, min_distance = distance((x_pos, y_pos), support_points)
                    if min_distance <= support_distance:
                        pos_z = support_focus[idx]
                        pos_z = limit_stage(pos_z, (config['hard-limit-z'][0], config['hard-limit-z'][1]), default=config["Z-stage-20x"])
                        core.set_position(pos_z)
                        core.wait_for_device(z_device)
                        pixels = self.snap_image(rgb=True, flip_channel=True)
                        bg_flag = is_background(pixels, t=bg_t, tt=bg_tt)
                    else:
                        pos_z, pixels, bg_flag = self.autofocus(mag='20x', rgb=True, search_range=120, steps=4, snap=True, preset=z_pos) # snap at top but return center z
                        if bg_flag:
                            if len(support_points)>=2:
                                pos_z = limit_stage(pos_z, (config['hard-limit-z'][0], config['hard-limit-z'][1]), default=config["Z-stage-20x"])
                                core.set_position(pos_z)
                            else:
                                pos_z = z_pos
                            core.wait_for_device(z_device)
                            pixels = self.snap_image(rgb=True, flip_channel=True)
                        else:
                            pos_z, pixels, _ = self.autofocus(mag='20x', rgb=True, search_range=42, steps=7, snap=True, check_background=False, preset=z_pos) 
                            support_points.append((x_pos, y_pos))
                            support_focus.append(pos_z)
                            sp_flag = True
                    z_positions[pos] = pos_z 
                    
                pixels = img_as_float(pixels)   
                
                if estimate_background:
                    if focus_dive:
                        bg_flag = bg_flag
                    else:
                        bg_flag = is_background(pixels, t=0.28, tt=0.28)
                        print('hard check')
                    if bg_flag:
                        print(' (background tile)')
                        redive_flag=True
                        bg_stack.append(pixels)
                    else:
                        redive_flag=False                
                if background_image is not None and not estimate_background:
                    pixels = white_balance(pixels, background_image)
                    pixels = flat_field(pixels, bg_img)
                    
                if plot_on:
                    show.set_data(pixels)
                    display.display(plt.gcf())
                    display.clear_output(wait=True)
                ### Use tifile to write out a tile with metadata?
                io.imsave(f'{acq_path}/{pos}-{bg_flag}-{sp_flag}.tif', img_as_ubyte(pixels), check_contrast=False)
                tile_count = tile_count + 1
                #sys.stdout.write('\r {}/{} tiles done'.format(tile_count, position_list.shape[0]))
                print(f"QuPath: {tile_count}/{position_list.shape[0]} tiles done")
                #plt.close('all')
            
        if position_list.shape[1] == 2:
            tile_count = 0
            core.set_focus_device(config['focus-device'])
            z_positions=np.ones(position_list.shape[0]) * core.get_position()
            for pos in range(position_list.shape[0]):
                x_pos = position_list[pos, 0]
                y_pos = position_list[pos, 1]
                
                x_pos = limit_stage(x_pos, (config['hard-limit-x'][0], config['hard-limit-x'][1]), default=None)
                y_pos = limit_stage(y_pos, (config['hard-limit-y'][0], config['hard-limit-y'][1]), default=None)
                    
                xy_device = core.get_xy_stage_device()
                z_device = core.get_focus_device()
                core.set_xy_position(x_pos, y_pos)
                core.wait_for_device(xy_device)
                sp_flag = False
                    
                if focus_dive and mag=='4x':
                    support_distance = config['pixel-size-bf-4x'] * config['camera-resolution'][1] * config['autofocus-speed']
                    idx, min_distance = distance((x_pos, y_pos), support_points)
                    if min_distance <= support_distance:
                        pos_z = support_focus[idx]
                        pos_z = limit_stage(pos_z, (config['hard-limit-z'][0], config['hard-limit-z'][1]), default=config['Z-stage-4x'])
                        core.set_position(pos_z)
                        core.wait_for_device(z_device)
                        pixels = self.snap_image(rgb=True, flip_channel=True)
                        bg_flag = is_background(pixels, t=bg_t, tt=bg_tt)
                    else:
                        pos_z, pixels, bg_flag = self.autofocus(mag='4x', rgb=True, search_range=100, steps=3, snap=True) # snap at top but return center z
                        if not bg_flag:
                            support_points.append((x_pos, y_pos))
                            support_focus.append(pos_z)
                            sp_flag = True
                    z_positions[pos] = pos_z
                    
                if focus_dive and mag=='20x':
                    support_distance = config["pixel-size-bf-20x"] * config["camera-resolution"][1] * config["autofocus-speed"]
                    idx, min_distance = distance((x_pos, y_pos), support_points)
                    if min_distance <= support_distance:
                        pos_z = support_focus[idx]
                        pos_z = limit_stage(pos_z, (config['hard-limit-z'][0], config['hard-limit-z'][1]), default=config['Z-stage-20x'])
                        core.set_position(pos_z)
                        core.wait_for_device(z_device)
                        pixels = self.snap_image(rgb=True, flip_channel=True)
                        bg_flag = is_background(pixels, t=bg_t, tt=bg_tt)
                    else:
                        pos_z, pixels, bg_flag = self.autofocus(mag='20x', rgb=True, search_range=50, steps=3, snap=False) # snap at top but return center z
                        pos_z, pixels, bg_flag = self.autofocus(mag='20x', rgb=True, search_range=10, steps=3, snap=True) # snap at top but return center z
                        if not bg_flag:
                            support_points.append((x_pos, y_pos))
                            support_focus.append(pos_z)
                            sp_flag = True
                    z_positions[pos] = pos_z    
                    
                pixels = img_as_float(pixels)                   
                
                if estimate_background:
                    if focus_dive:
                        bg_flag = bg_flag
                    else:
                        bg_flag = is_background(pixels, t=bg_t, tt=bg_tt)
                        print('hard check')
                    if bg_flag:
                        print(' (background tile)')
                        redive_flag=True
                        bg_stack.append(pixels)
                    else:
                        redive_flag=False    
                                    
                if background_image is not None and not estimate_background:
                    pixels = white_balance(pixels, background_image)
                    pixels = flat_field(pixels, bg_img)
                    
                if plot_on:
                    show.set_data(pixels)
                    display.display(plt.gcf())
                    display.clear_output(wait=True)
                ### Use tifile to create tile with metadata?
                io.imsave(acq_path+'/{}-{}-{}.tif'.format(pos, bg_flag, sp_flag), img_as_ubyte(pixels), check_contrast=False)
                tile_count = tile_count + 1
                #sys.stdout.write('\r {}/{} tiles done'.format(tile_count, position_list.shape[0]))
                print(f"QuPath: {tile_count}/{position_list.shape[0]} tiles done")
                if plot_on:
                    plt.close('all')
        if estimate_background:
            if len(bg_stack)==0:
                if mag=='4x':
                    background_image = self.bf_4x_bg
                    if self.bf_process_fn is not None: background_image = self.bf_process_fn(background_image)
                elif mag=='20x':
                    background_image = self.bf_20x_bg
                    if self.bf_process_fn is not None: background_image = self.bf_process_fn(background_image)
                returns['Background image'] = background_image
                # io.imsave(acq_path+'/bg_img.tif', img_as_ubyte(background_image))
            else:
                bg_stack= np.stack(bg_stack)
                median = np.median(bg_stack, axis=0)
                median = img_as_float(median)
                returns['Background image'] = median
                # io.imsave(acq_path+'/bg_img.tif', img_as_ubyte(median))
        else:
            returns['Background image'] = background_image
        if focus_dive:
            z_positions = z_positions.reshape(position_list.shape[0], 1)
            returns['Z positions'] = z_positions
        return returns

In [ ]:
acq_name = sampleLabel + "-4x-bf"